# Regression Example: Depth
This notebook is for illustrating the effects of adversarial transformers on a progressive learner for regression. We will run toy experiments with a normally trained transformer and an adversarial transformer in the same progressive learner using a SimpleAverage decider. With this toy example, we train the normal transformer on the data y = x and will be iterating over a depth of 1-30 with 10 trees.

We will run 3 experiments using an adversarial transformer:
1. that outputs random partitions
2. that is trained on y = 0 data
3. that is trained on very noisy y = x data

To show that adversarial transformers are bad, we would expect the generalization error of the data predicted by the adversarial transformer to be higher than that of the normal transformer.

In [6]:
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import seaborn as sns
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from math import log2, ceil

In [7]:
from proglearn.progressive_learner import ProgressiveLearner
from proglearn.deciders import SimpleAverage, LinearRegressionDecider
from proglearn.transformers import TreeRegressionTransformer
# from proglearn.transformers import RandomTransformer
from proglearn.voters import TreeRegressionVoter
from joblib import Parallel, delayed

# Creates the experimental data.

In [8]:
def createExperimentData(sample_size, exp2=False):
    # Create the Task A dataset
    X, y, coef = make_regression(n_samples=sample_size, n_features=1, n_informative=1, noise=0.1, coef=True, bias=0.0)

    # Use linear regression to find the true values
    regressor = LinearRegression()
    regressor.fit(X, y)
    
    # Rescale y = mx + b to be y* = x, where y* = (y - b) / m
    y = (y - regressor.intercept_) / regressor.coef_
    y = y.reshape(X.shape[0], -1)
    
    task_A_data = np.append(X, y, axis=1)
    
    # Test B data
    if exp2==True:
        X_new, y_new = make_regression(n_samples=sample_size, n_features=1, n_informative=1, noise=0.1, bias=0.0)
        regressor.fit(X_new, y_new)
        y_new = np.zeros((X_new.shape[0], 1))
    elif exp2==False:
        X_new, y_new = make_regression(n_samples=int(sample_size / 10), n_features=1, n_informative=1, noise=10.0, bias=0.0)
        regressor.fit(X_new, y_new)
        y_new = (y_new - regressor.intercept_) / regressor.coef_
        y_new = y_new.reshape(X_new.shape[0], -1)
        y_new += np.random.normal(size=y_new.shape, scale=20.0)
        plt.plot(X_new, y_new)

    task_B_data = np.append(X_new, y_new, axis=1)
    
    return task_A_data, task_B_data

# Experiment Setup

In [9]:
def experiment(n_task_A, n_task_B, n_test=0.25, n_trees_A=25, n_trees_B=25, n_depth_A=30, 
               n_depth_B=30, zeros=False, randomTransformer=False, acorn=None):
    
    if acorn != None:
        np.random.seed(acorn)
        
    errors = np.zeros(3, dtype=float)
    
    transformer_voter_decider_split = [0.67, 0.33, 0]
    
    default_transformer_class = TreeRegressionTransformer
    taskA_transformer_kwargs = {"kwargs": {"max_depth": n_depth_A}}
    taskB_transformer_kwargs = {"kwargs": {"max_depth": n_depth_B}}

    default_voter_class = TreeRegressionVoter
    default_voter_kwargs = {}

    default_decider_class = SimpleAverage
    default_decider_kwargs = {}

    progressive_learner = ProgressiveLearner(default_transformer_class = default_transformer_class, 
                                             default_transformer_kwargs = taskA_transformer_kwargs,
                                             default_voter_class = default_voter_class,
                                             default_voter_kwargs = default_voter_kwargs,
                                             default_decider_class = default_decider_class,
                                             default_decider_kwargs = default_decider_kwargs)

    # Create data
    taskA_data, taskB_data = createExperimentData(n_task_A, zeros)
    
    # Split task data into train/test
    taskA_x_train, taskA_x_test, taskA_label_train, taskA_label_test = train_test_split(taskA_data[:,0], 
                                                                                        taskA_data[:,1], 
                                                                                        test_size=n_test, 
                                                                                        random_state=7)

    taskB_x_train, taskB_x_test, taskB_label_train, taskB_label_test = train_test_split(taskB_data[:,0], 
                                                                                        taskB_data[:,1], 
                                                                                        test_size=n_test, 
                                                                                        random_state=7)
    
    taskA_x_train = taskA_x_train.reshape(-1, 1)
    taskA_x_test = taskA_x_test.reshape(-1, 1)
    taskA_label_train = taskA_label_train.reshape(-1, 1)
    taskA_label_test = taskA_label_test.reshape(-1, 1)

    taskB_x_train = taskB_x_train.reshape(-1, 1)
    taskB_x_test = taskB_x_test.reshape(-1, 1)
    taskB_label_train = taskB_label_train.reshape(-1, 1)
    taskB_label_test = taskB_label_test.reshape(-1, 1)

    if (n_task_A == 0):
        progressive_learner.add_task(taskB_x_train, taskB_label_train)
        l2f_taskB = progressive_learner.predict(taskB_x_test, transformer_ids=[0], task_id=0)

        errors[0] = 0.5
        errors[1] = mean_squared_error(lf2_taskB, taskB_label_test)
        
    elif (n_task_B == 0):
        progressive_learner.add_task(taskA_x_train, taskA_label_train)
        l2f_taskA = progressive_learner.predict(taskA_x_test, transformer_ids=[0], task_id=0)

        errors[0] = mean_squared_error(l2f_taskA, taskA_label_test)
        errors[1] = 0.5
        
    else:
        # Add tasks to progressive learner
        progressive_learner.add_task(taskA_x_train, taskA_label_train)
        
        # Predict and record loss without adversarial transformer
        l2f_taskA = progressive_learner.predict_proba(taskA_x_test, transformer_ids=[0], task_id=0)
        errors[0] = mean_squared_error(l2f_taskA, taskA_label_test)
        
        # Create adversarial transformer
        if (randomTransformer):
            progressive_learner.add_transformer(taskB_x_train, taskB_label_train,
                                                transformer_class=TreeRegressionRandomTransformer,
                                                transformer_kwargs=taskB_transformer_kwargs,
                                                voter_class=default_voter_class,
                                                voter_kwargs=default_voter_kwargs,
                                                transformer_id=1
                                               )
        else:
            progressive_learner.add_transformer(taskB_x_train, taskB_label_train,
                                                transformer_class=default_transformer_class,
                                                transformer_kwargs=taskB_transformer_kwargs,
                                                voter_class=default_voter_class,
                                                voter_kwargs=default_voter_kwargs,
                                                transformer_id=1
                                               )

        # Predict and record loss with adversarial transformer
        l2f_taskA_adversarial = progressive_learner.predict_proba(taskA_x_test, transformer_ids=[0, 1], task_id=0)
        errors[1] = mean_squared_error(l2f_taskA_adversarial, taskA_label_test)
        
        # Predict with inference through adversarial 
        l2f_taskA_adversarial_only = progressive_learner.predict_proba(taskA_x_test, transformer_ids=[1], task_id=0)
        errors[2] = mean_squared_error(l2f_taskA_adversarial_only, taskA_label_test)
        
    return errors

# Experiment 1

In [3]:
# Set up the tree parameters
mc_rep = 100
n_test = 0.25
n_trees = 25
n_sample_size = 1000
max_depth = 30

range_depths = (100*np.arange(0.01, 0.31, step=0.01)).astype(int)

# Initiate error arrays
mean_error = np.zeros((3, len(range_depths)))
std_error = np.zeros((3, len(range_depths)))

# Initiate transfer efficiencies
mean_te = np.zeros((1, len(range_depths)))
std_te = np.zeros((1, len(range_depths)))

# Iterate over the depths
for i, n1 in enumerate(range_depths):
    print('starting to compute, depth=%s\n' %n1)
    error = np.array(
        Parallel(n_jobs=-1, verbose=1)(
            delayed(experiment)(
                n_sample_size, n_sample_size, n_test=n_test, n_trees_A=n_trees, 
                n_trees_B=n_trees, n_depth_A=max_depth, n_depth_B=n1, zeros=True
            ) for _ in range(mc_rep)
        )
    )
    mean_error[:, i] = np.mean(error, axis=0)
    std_error[:, i] = np.std(error, ddof=1, axis=0)
    
    mean_te[0, i] = np.mean(error[:, 0]) / np.mean(error[:, 1])
        

with open('./data/mean_setting2.pickle', 'wb') as f:
    pickle.dump(mean_error, f)
    
with open('./data/std_setting2', 'wb') as f:
    pickle.dump(std_error, f)

with open('./data/mean_te_setting2', 'wb') as f:
    pickle.dump(mean_te, f)

starting to compute, depth=1



NameError: name 'Parallel' is not defined

# Experiment 2
We will use an adversarial transformer trained on y = 0 data and a normal transformer trained on y = x data.

In [10]:
# Set up the tree parameters
mc_rep = 100
n_test = 0.25
n_trees = 25
n_sample_size = 1000
max_depth = 30

range_depths = (100*np.arange(0.01, 0.31, step=0.01)).astype(int)

# Initiate error arrays
mean_error = np.zeros((3, len(range_depths)))
std_error = np.zeros((3, len(range_depths)))

# Initiate transfer efficiencies
mean_te = np.zeros((1, len(range_depths)))
std_te = np.zeros((1, len(range_depths)))

# Iterate over the depths
for i, n1 in enumerate(range_depths):
    print('starting to compute, depth=%s\n' %n1)
    error = np.array(
        Parallel(n_jobs=-1, verbose=1)(
            delayed(experiment)(
                n_sample_size, n_sample_size, n_test=n_test, n_trees_A=n_trees, 
                n_trees_B=n_trees, n_depth_A=max_depth, n_depth_B=n1, zeros=True
            ) for _ in range(mc_rep)
        )
    )
    mean_error[:, i] = np.mean(error, axis=0)
    std_error[:, i] = np.std(error, ddof=1, axis=0)
    
    mean_te[0, i] = np.mean(error[:, 0]) / np.mean(error[:, 1])
        

with open('./data/mean_setting2.pickle', 'wb') as f:
    pickle.dump(mean_error, f)
    
with open('./data/std_setting2', 'wb') as f:
    pickle.dump(std_error, f)

with open('./data/mean_te_setting2', 'wb') as f:
    pickle.dump(mean_te, f)

starting to compute, depth=1



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   12.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, depth=2



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  93 out of 100 | elapsed:    4.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, depth=3



...

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, depth=29



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  93 out of 100 | elapsed:    5.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, depth=30



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.9s finished


# Experiment 3
Adversarial transformer trained on noisy, low-sampled, y=x data

In [ ]:
# Set up the tree parameters
mc_rep = 100
n_test = 0.25
n_trees = 25
n_sample_size = 1000
max_depth = 30

range_depths = (100*np.arange(0.01, 0.31, step=0.01)).astype(int)

# Initiate error arrays
mean_error = np.zeros((3, len(range_depths)))
std_error = np.zeros((3, len(range_depths)))

# Initiate transfer efficiencies
mean_te = np.zeros((1, len(range_depths)))
std_te = np.zeros((1, len(range_depths)))

# Iterate over the depths
for i, n1 in enumerate(range_depths):
    print('starting to compute, depth=%s\n' %n1)
    error = np.array(
        Parallel(n_jobs=-1, verbose=1)(
            delayed(experiment)(
                n_sample_size, n_sample_size, n_test=n_test, n_trees_A=n_trees, 
                n_trees_B=n_trees, n_depth_A=max_depth, n_depth_B=n1, zeros=False
            ) for _ in range(mc_rep)
        )
    )
    mean_error[:, i] = np.mean(error, axis=0)
    std_error[:, i] = np.std(error, ddof=1, axis=0)
    
    mean_te[0, i] = np.mean(error[:, 0]) / np.mean(error[:, 1])
        

with open('./data/mean_setting3.pickle', 'wb') as f:
    pickle.dump(mean_error, f)
    
with open('./data/std_setting3', 'wb') as f:
    pickle.dump(std_error, f)

with open('./data/mean_te_setting3', 'wb') as f:
    pickle.dump(mean_te, f)

starting to compute, depth=1



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  93 out of 100 | elapsed:    2.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, depth=2



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, depth=3



...

starting to compute, depth=15



[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, depth=16



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, depth=17



# Experiment 4

In [12]:
# Set up the tree parameters
mc_rep = 100
n_test = 0.25
n_trees = 25
n_sample_size = 1000
max_depth = 30

range_trees = (100*np.arange(0.05, 0.55, step=0.05)).astype(int)

# Initiate error arrays
mean_error = np.zeros((3, len(range_trees)))
std_error = np.zeros((3, len(range_trees)))

# Initiate transfer efficiencies
mean_te = np.zeros((1, len(range_trees)))
std_te = np.zeros((1, len(range_trees)))

# Iterate over the depths
for i, n1 in enumerate(range_trees):
    print('starting to compute, tree=%s\n' %n1)
    error = np.array(
        Parallel(n_jobs=-1, verbose=1)(
            delayed(experiment)(
                n_sample_size, n_sample_size, n_test=n_test, n_trees_A=n_trees, 
                n_trees_B=n1, n_depth_A=max_depth, n_depth_B=max_depth, zeros=True
            ) for _ in range(mc_rep)
        )
    )
    mean_error[:, i] = np.mean(error, axis=0)
    std_error[:, i] = np.std(error, ddof=1, axis=0)
    
    mean_te[0, i] = np.mean(error[:, 0]) / np.mean(error[:, 1])
        

with open('./data/mean_setting4.pickle', 'wb') as f:
    pickle.dump(mean_error, f)
    
with open('./data/std_setting4', 'wb') as f:
    pickle.dump(std_error, f)

with open('./data/mean_te_setting4', 'wb') as f:
    pickle.dump(mean_te, f)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, tree=5



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  93 out of 100 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, tree=10



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  93 out of 100 | elapsed:    2.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, tree=45



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  93 out of 100 | elapsed:    2.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, tree=50



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.5s finished


# Experiment 5

In [13]:
# Set up the tree parameters
mc_rep = 100
n_test = 0.25
n_trees = 25
n_sample_size = 1000
max_depth = 30

range_trees = (100*np.arange(0.05, 0.55, step=0.05)).astype(int)

# Initiate error arrays
mean_error = np.zeros((3, len(range_trees)))
std_error = np.zeros((3, len(range_trees)))

# Initiate transfer efficiencies
mean_te = np.zeros((1, len(range_trees)))
std_te = np.zeros((1, len(range_trees)))

# Iterate over the depths
for i, n1 in enumerate(range_trees):
    print('starting to compute, tree=%s\n' %n1)
    error = np.array(
        Parallel(n_jobs=-1, verbose=1)(
            delayed(experiment)(
                n_sample_size, n_sample_size, n_test=n_test, n_trees_A=n_trees, 
                n_trees_B=n1, n_depth_A=max_depth, n_depth_B=max_depth, zeros=True
            ) for _ in range(mc_rep)
        )
    )
    mean_error[:, i] = np.mean(error, axis=0)
    std_error[:, i] = np.std(error, ddof=1, axis=0)
    
    mean_te[0, i] = np.mean(error[:, 0]) / np.mean(error[:, 1])
        

with open('./data/mean_setting5.pickle', 'wb') as f:
    pickle.dump(mean_error, f)
    
with open('./data/std_setting5', 'wb') as f:
    pickle.dump(std_error, f)

with open('./data/mean_te_setting5', 'wb') as f:
    pickle.dump(mean_te, f)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, tree=5



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, tree=10



[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  93 out of 100 | elapsed:    2.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, tree=45



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


starting to compute, tree=50



[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  93 out of 100 | elapsed:    2.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.2s finished


# Plots

In [ ]:
with open('data/mean_setting2.pickle','rb') as f:
    mean_error_2 = pickle.load(f)
    
with open('data/mean_setting3.pickle','rb') as f:
    mean_error_3 = pickle.load(f)
    
with open('data/mean_setting4.pickle','rb') as f:
    mean_error_4 = pickle.load(f)
    
with open('data/mean_setting5.pickle','rb') as f:
    mean_error_5 = pickle.load(f)

range_depths = (100*np.arange(0.01, 0.31, step=0.01)).astype(int)
range_trees = (100*np.arange(0.05, 0.55, step=0.05)).astype(int)

fig, ax = plt.subplots(2,2, figsize=(26,20))

ax[0,0].scatter(range_depths, mean_error_2[0,:], s=10**2, label='Task A with Normal')
ax[0,0].scatter(range_depths, mean_error_2[1,:], s=10**2, label='Task A with Normal + Adverserial')
ax[0,0].scatter(range_depths, mean_error_2[2,:], s=10**2, label='Task A with Adversarial')
ax[0,0].set_xlabel('Depth', fontsize=25)
ax[0,0].set_ylabel('Mean MSE', fontsize=25)
ax[0,0].set_title('Adversarial Transformer trained on y = 0', fontsize=25)
ax[0,0].tick_params(labelsize=25)

ax[0,1].scatter(range_depths, mean_error_3[0,:], s=10**2, label='Task A with Normal')
ax[0,1].scatter(range_depths, mean_error_3[1,:], s=10**2, label='Task A with Normal + Adverserial')
ax[0,1].scatter(range_depths, mean_error_3[2,:], s=10**2, label='Task A with Adversarial')
ax[0,1].legend(loc='right', frameon=False, bbox_to_anchor=(1.8, 0.5), fontsize=25)
ax[0,1].set_xlabel('Depth', fontsize=25)
ax[0,1].set_ylabel('Mean MSE', fontsize=25)
ax[0,1].set_title('Adversarial Transformer trained on noisy y = x', fontsize=25)
ax[0,1].tick_params(labelsize=25)

ax[1,0].scatter(range_trees, mean_error_4[0,:], s=10**2, label='Task A with Normal')
ax[1,0].scatter(range_trees, mean_error_4[1,:], s=10**2, label='Task A with Normal + Adverserial')
ax[1,0].scatter(range_trees, mean_error_4[2,:], s=10**2, label='Task A with Adversarial')
ax[1,0].set_xlabel('Trees', fontsize=25)
ax[1,0].set_ylabel('Mean MSE', fontsize=25)
ax[1,0].set_title('Adversarial Transformer trained on y = 0', fontsize=25)
ax[1,0].tick_params(labelsize=25)

ax[1,1].scatter(range_trees, mean_error_5[0,:], s=10**2, label='Task A with Normal')
ax[1,1].scatter(range_trees, mean_error_5[1,:], s=10**2, label='Task A with Normal + Adverserial')
ax[1,1].scatter(range_trees, mean_error_5[2,:], s=10**2, label='Task A with Adversarial')
ax[1,1].set_xlabel('Trees', fontsize=25)
ax[1,1].set_ylabel('Mean MSE', fontsize=25)
ax[1,1].set_title('Adversarial Transformer trained on noisy y = x', fontsize=25)
ax[1,1].tick_params(labelsize=25)